### Dataset basic operations

In [1]:
import sys
import os
import zipfile
import json
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.articles import process_raw_article

Setup the paths to data sources

In [2]:
ARCHIVE_FILENAME = "2025_02_25_wienerzeitung_archiv.zip"
ARCHIVE_EXTRACT_DIR = os.path.join("..", "data", "articles_raw")
ARTICLES_CLEAN_DIR = os.path.join("..", "data", "articles_clean")
ARTICLES_RAW_DIR = os.path.join("..", "data", "articles_raw", "2025_02_25_wienerzeitung_archiv", "content")
archive_path = os.path.join("..", "data", "archive", ARCHIVE_FILENAME)

Extract ZIP archive to raw JSONs

In [4]:
# Extract the archive to the articles directory
with zipfile.ZipFile(archive_path, "r") as archive:
    archive.extractall(ARCHIVE_EXTRACT_DIR)

KeyboardInterrupt: 

In [5]:
print(f"Number of articles in archive: {len(os.listdir(ARTICLES_RAW_DIR))}")

Number of articles in archive: 87754


Clean the raw JSON to key content

In [7]:
def process_article_file(article_file):
    article_path = os.path.join(ARTICLES_RAW_DIR, article_file)
    # Read the raw article
    with open(article_path, "r", encoding="utf-8") as file:
        article = json.load(file)
    
    # Process the article
    article_clean = process_raw_article(article)
    
    # Write the cleaned article
    clean_path = os.path.join(ARTICLES_CLEAN_DIR, article_file)
    with open(clean_path, "w", encoding="utf-8") as file:
        json.dump(article_clean, file, indent=4, ensure_ascii=False)
    return article_file

# Ensure the output directory exists
if not os.path.exists(ARTICLES_CLEAN_DIR):
    os.makedirs(ARTICLES_CLEAN_DIR)

# List of raw article files
articles_raw = os.listdir(ARTICLES_RAW_DIR)

# Use ThreadPoolExecutor for concurrent processing
with ThreadPoolExecutor(max_workers=8) as executor:
    futures = {executor.submit(process_article_file, article_file): article_file for article_file in articles_raw}
    
    for future in tqdm(as_completed(futures), total=len(futures)):
        try:
            file_name = future.result()
        except Exception as e:
            print(f"Error processing {futures[future]}: {e}")


100%|██████████| 87754/87754 [00:35<00:00, 2443.97it/s]


Example of raw article

In [8]:
# Pick 1st article for showcase
raw_article_path = os.path.join(ARTICLES_RAW_DIR, articles_raw[0])
with open(raw_article_path, "r", encoding="utf-8") as file:
    raw_article = json.load(file)
raw_article

{'name': '007 und die Lizenz zum Verspießern',
 'uuid': '56b42350-1307-421b-8394-5fa2d55f21f5',
 'full_slug': 'h/007-und-die-lizenz-zum-verspiessern',
 'slug': '007-und-die-lizenz-zum-verspiessern',
 'first_published_at': '2017-01-17 16:32',
 'content': {'component': 'archiv-inhalt',
  'titel': '007 und die Lizenz zum Verspießern',
  'excerpt': {'type': 'doc', 'content': []},
  'autor': 'Christoph Irrgeher',
  'oldId': 868490,
  'category': {'name': 'Kommentare',
   'full_slug': 'hr/kommentare',
   'content': {'parent': 'Meinung'}},
  'inhalt': {'type': 'doc',
   'content': [{'type': 'paragraph',
     'content': [{'text': 'Neuseeländische Forscher schlagen Alarm: James Bond, heißt es in einem Aufsatz, der sich den Agenten in der "echten" Welt vorstellt, sei massiv bedroht.',
       'type': 'text',
       'marks': []},
      {'type': 'hard_break'},
      {'type': 'hard_break'},
      {'text': 'Das wirkt nicht neu. Immerhin ist 007 chronisch von Superbösen umgeben - Düsterlingen, die den

Example of cleaned article

In [17]:
articles_clean = os.listdir(ARTICLES_CLEAN_DIR)
sample_article = articles_clean[1000]
sample_article_path = os.path.join(ARTICLES_CLEAN_DIR, sample_article)

with open(sample_article_path, "r", encoding="utf-8") as file:
    sample_article = json.load(file)

for key, value in sample_article.items():
    print(f"{key}: {value} \n")

id: 0922f937-1922-404f-9e9e-27c066084743 

published_at: 2023-04-14 16:27 

author: Walter Hämmerle 

title: Absage an den Endkampf 

category: Leitartikel 

ressort: Meinung 

text: Weil jetzt ziemlich viele Menschen - oder besser gesagt: viele Linke, die sich oft und lang in den Sozialen Medien aufhalten - begeistert sind von der Art, wie Andreas Babler seine Ideen für Politik kommuniziert: Die Moderne beginnt als Erzählung von Selbstermächtigung und Anmaßung, heute wird sie von vielen als Gefühl der Überforderung erlebt, bei den sachlichen Herausforderungen, aber durchaus auch beim Umgang der Medien mit diesen Themen. "Overnewsed but underinformed", und das schon seit Jahrzehnten.
Von der Politik erwarten sich trotzdem die meisten Menschen, dass sie das Steuerrad der ständigen Veränderung fest in Händen hält. Kein Wunder, bemüht sich die Politik doch nach Kräften, exakt diesen Eindruck zu vermitteln, dabei ist sie selbst Getriebene der Entwicklungen von unten wie oben, auf die es zu